## Data Wrangling: Clean, Transform, Merge, Reshape

In [2]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas
import pandas as pd
np.set_printoptions(precision=4, threshold=500)
pd.options.display.max_rows = 100

In [3]:
%matplotlib inline

## 데이터셋 결합 및 병합

### DB-스타일 데이터프레임 병합

In [4]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'],
                 'data2': range(3)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [5]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [6]:
pd.merge(df1, df2)
# 디폴트로 겹치는 컬럼이름을 기준으로 한다

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [7]:
pd.merge(df1, df2, on='key')
# 명시적으로 지정하는 것이 안전하다

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [8]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'],
                 'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey') #merge함수는 중복된 값을 기준으로 해서 병합한다

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [9]:
pd.merge(df4, df3, left_on='rkey', right_on='lkey') #merge함수는 중복된 값을 기준으로 해서 병합한다

,data2,rkey,data1,lkey
0,0,a,2,a
1,0,a,4,a
2,0,a,5,a
3,1,b,0,b
4,1,b,1,b
5,1,b,6,b


### 실습 예제 1
컬럼명이 나이, 이름 두 가지로 구성된 데이터프레임 df_ex1, df_ex2를 생성하고, 나이를 기준으로 merge하여 출력하여라.(단, 한 프레임 당 행의 수는 6개 이상) 

In [10]:
df_ex1 = DataFrame({'name': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'ag3': range(7)})

In [11]:
df_ex2 = DataFrame({'name': ['a', 'b', 'd'],
                 'age': range(3)})

In [12]:
pd.merge(df_ex1, df_ex2, on='name')

,ag3,name,age
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [13]:
data = Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [14]:
#치환하는 함수 replace
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [10]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [15]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [16]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 실습예제 2
0부터 10까지의 Series 자료구조를 만든 후 5 이상의 수를 Nan으로 replace하여라.

In [17]:
a = Series(range(11))
a
for i in a:
    if i >=5:
        a = a.replace(i, np.nan)
a

0     0.0
1     1.0
2     2.0
3     3.0
4     4.0
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
dtype: float64

In [18]:
se = Series(range(11))
for i in se:
    if i >= 5:
        se = se.replace(i, np.nan)
se

0     0.0
1     1.0
2     2.0
3     3.0
4     4.0
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
dtype: float64

### 이상치 필터링 및 검출

In [19]:
np.random.seed()
data = DataFrame(np.random.randn(1000, 4)) #(row, col)
data

,0,1,2,3
0,1.291922,0.793841,0.604602,1.259937
1,0.252012,0.310706,0.211441,1.383343
2,0.130544,0.920621,-0.296376,-0.776104
3,-1.102923,-0.023149,-1.434832,1.352448
4,-0.004134,1.561814,-0.486476,0.912792
5,0.183907,0.123978,-0.778762,0.144082
6,-0.157361,0.423752,1.253219,0.837937
7,-1.174373,-1.136332,-0.482916,-0.109644
8,0.050431,-1.228553,0.510349,0.746816
9,-0.847861,0.828586,0.179533,-0.271776


In [20]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.028098,-0.061430,0.001573,-0.028143
std,1.003055,0.988331,1.002936,0.983721
min,-3.312677,-4.228997,-3.206421,-4.010758
25%,-0.638954,-0.728534,-0.682096,-0.666608
50%,0.011222,-0.057375,-0.005125,-0.035729
75%,0.739440,0.599120,0.662499,0.635965
max,3.780899,2.915427,3.165153,3.197240


In [21]:
col = data[3]

In [22]:
col[np.abs(col) > 3] #절대값을 구하는 함수 abs()

140   -4.010758
312    3.050241
673   -3.001982
853    3.197240
Name: 3, dtype: float64

In [23]:
data[(np.abs(data) > 3).any(1)] #데이터프레임의 행의 1개 이상의 원소에 절대값을 씌웠을 때 3보다 큰 값이 포함된 행이 있으면 그 행을 출력.

,0,1,2,3
68,-3.312677,-0.479027,-0.356450,0.127019
140,1.808982,-0.871143,0.540171,-4.010758
231,3.085370,-0.704910,0.880059,-0.049124
300,3.780899,0.436385,-1.463147,0.988522
312,-1.414525,0.643525,-0.951308,3.050241
410,0.570582,-4.228997,-0.557842,1.314426
454,2.376614,-3.153635,-0.612053,-0.602248
561,-0.402267,1.738268,3.165153,-0.624122
673,-0.000156,1.270047,-0.091181,-3.001982
747,0.189626,0.844238,-3.206421,0.694828


In [24]:
data[np.abs(data) > 3] = np.sign(data) * 3 #절대값을 취한 것이 3보다 큰 값을 가진 데이터프레임 data에 sign함수를 취함

In [25]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.027545,-0.059134,0.001614,-0.027378
std,0.999177,0.980338,1.001788,0.979345
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.638954,-0.728534,-0.682096,-0.666608
50%,0.011222,-0.057375,-0.005125,-0.035729
75%,0.739440,0.599120,0.662499,0.635965
max,3.000000,2.915427,3.000000,3.000000


### 실습예제 3
위의 방법을 응용해 100행 5열의 난수로 구성된 데이터프레임을 생성하고 2번 열의 데이터가 음수인 것만 출력하여라.

In [26]:
np.random.seed(1)
data_ex1 = DataFrame(np.random.randn(100, 5)) #(row, col)
data_ex1
col = data_ex1[2]
col[col < 0] #절대값을 구하는 함수 abs()

0    -0.528172
1    -0.761207
2    -0.322417
3    -0.877858
5    -0.935769
6    -0.687173
8    -0.747158
10   -1.142518
18   -0.375285
21   -0.953701
23   -1.615772
26   -1.230058
27   -1.144341
31   -0.119054
35   -0.287308
40   -0.562305
41   -0.890556
43   -0.434957
45   -0.198838
47   -0.665755
48   -0.001891
49   -0.443172
50   -2.793085
60   -0.830172
61   -0.694694
63   -0.051188
66   -0.997020
67   -1.942589
68   -0.442229
69   -0.174180
70   -0.195057
71   -0.190221
72   -1.943323
73   -2.782534
75   -0.785534
76   -1.538793
77   -0.175459
78   -0.475373
81   -0.830169
84   -0.374805
88   -1.160170
94   -0.198998
95   -1.402329
96   -0.767310
98   -0.414847
99   -2.237087
Name: 2, dtype: float64

In [56]:
data[data < 3]

,0,1,2,3
0,1.624345,-0.611756,-0.528172,-1.072969
1,0.865408,-2.301539,1.744812,-0.761207
2,0.319039,-0.249370,1.462108,-2.060141
3,-0.322417,-0.384054,1.133769,-1.099891
4,-0.172428,-0.877858,0.042214,0.582815
5,-1.100619,1.144724,0.901591,0.502494
6,0.900856,-0.683728,-0.122890,-0.935769
7,-0.267888,0.530355,-0.691661,-0.396754
8,-0.687173,-0.845206,-0.671246,-0.012665
9,-1.117310,0.234416,1.659802,0.742044


### 순열 및 랜덤 샘플링

In [17]:
df = DataFrame(np.arange(5 * 4).reshape(5, 4))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [18]:
len(df)

5

In [19]:
# 행의 위치를 랜덤하게 바꾸려고 한다
sampler = np.random.permutation(5)
sampler

array([1, 2, 4, 0, 3])

In [20]:
# 행을 선택할 때 take를 사용한다
df.take(sampler)

,0,1,2,3
1,4,5,6,7
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3
3,12,13,14,15


In [21]:
# 임의의 3개의 행만 추출하는 방법
df.take(np.random.permutation(len(df))[:3])

,0,1,2,3
0,0,1,2,3
4,16,17,18,19
1,4,5,6,7


In [22]:
# 15개의 정수 난수를 만들고 이 위치에 해당하는 데이터를 샘플링하는 방법
# bag에서 임의의 갯수를 추출하는 방법
bag = np.array([5, 7, -1, 6, 4])
sampler = np.random.randint(0, len(bag), size=15)

In [23]:
sampler

array([0, 0, 2, 3, 1, 2, 3, 4, 0, 0, 1, 4, 3, 1, 4])

In [24]:
draws = bag.take(sampler)
draws

array([ 5,  5, -1,  6,  7, -1,  6,  4,  5,  5,  7,  4,  6,  7,  4])

### 지표(key) 계산 및 쓰레기(더미)값

In [57]:
# 더미 변수를 만드는 방법
df = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1': range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [61]:
pd.get_dummies(df['key']) # 더미값을 얻어오는 함수 get_dummies

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [60]:
pd.get_dummies(df['data1'])

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,1,0,0,0
3,0,0,0,1,0,0
4,0,0,0,0,1,0
5,0,0,0,0,0,1


In [27]:
# 두개의 데이터프레임을 합치려면 join을 사용한다

dummies = pd.get_dummies(df['key'], prefix='asd')
dummies

NameError: name 'df' is not defined

In [67]:
df_with_dummy = dummies.join(df['data1'])
df_with_dummy

,key_a,key_b,key_c,data1
0,0,1,0,0
1,0,1,0,1
2,1,0,0,2
3,0,0,1,3
4,1,0,0,4
5,0,1,0,5


### 실습예제 4
생성된 위의 df_with_dummy 데이터프레임의 행의 위치를 랜덤하게 바꿔 출력하여라.

In [68]:
sampler = np.random.permutation(len(df_with_dummy))
df_with_dummy.take(sampler)

,key_a,key_b,key_c,data1
0,0,1,0,0
1,0,1,0,1
2,1,0,0,2
5,0,1,0,5
3,0,0,1,3
4,1,0,0,4


In [28]:
#개봉년도, 영화제목, 장르의 정보를 가진 20세기 영화 데이터
# 세개의 컬럼만 읽는다
mnames = ['movie_id', 'title', 'genres']

In [35]:
movies = pd.read_table('data/movies.dat', sep='::', header=None, names=mnames) 
movies[:10]

C:\Users\samsung\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [82]:
genre_iter = (set(x.split('|')) for x in movies.genres)
print(genre_iter)

<generator object <genexpr> at 0x00000045D268F410>


In [29]:
# 쟝르에서 유일한 값만 찾는다 set.union 사용
genres = sorted(set.union(*genre_iter))
print(genres)

<generator object <genexpr> at 0x000002242C34D620>
['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [30]:
dummies = DataFrame(np.zeros((len(movies), len(genres))), columns=genres)
# 0으로 초기화 한 데이터프레임

In [31]:
# 해당 쟝르가 있는 부분만 1로 표시한다
for i, gen in enumerate(movies.genres):
    dummies.ix[i, gen.split('|')] = 1
dummies[:10]

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [32]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
#더미값으로 해당하는 장르에 0과 1로 표현
movies_windic.ix[0:10]

,movie_id,title,genres,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Genre_Fantasy,Genre_Film-Noir,Genre_Horror,Genre_Musical,Genre_Mystery,Genre_Romance,Genre_Sci-Fi,Genre_Thriller,Genre_War,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,Heat (1995),Action|Crime|Thriller,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,7,Sabrina (1995),Comedy|Romance,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,8,Tom and Huck (1995),Adventure|Children's,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,Sudden Death (1995),Action,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,GoldenEye (1995),Action|Adventure|Thriller,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [33]:
np.random.seed(12345)

In [34]:
values = np.random.rand(10)
values

array([ 0.9296,  0.3164,  0.1839,  0.2046,  0.5677,  0.5955,  0.9645,
        0.6532,  0.7489,  0.6536])

In [35]:
# 데이터가 어느 범주에 속하는지 간단히 찾을 수 있다
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


### 실습예제 5
0~100 사이의 난수 10개를 생성하고 데이터가 10 단위의 범주 중 어느 범주에 속하는지를 구하여라

In [84]:
val = np.random.rand(10) * 100
ff = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
pd.get_dummies(pd.cut(val, ff))

,"(0, 10]","(10, 20]","(20, 30]","(30, 40]","(40, 50]","(50, 60]","(60, 70]","(70, 80]","(80, 90]","(90, 100]"
0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1,0,0,0
5,0,0,0,0,0,0,0,1,0,0
6,0,0,0,0,0,0,0,1,0,0
7,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,1,0,0,0
9,0,0,0,0,0,0,0,0,0,1


## 문자열 조작

### 문자열 메소드

In [37]:
# 파이선은 문자 열을 다루는데 매우 편리하다
# 문자열 구분하기

val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [86]:
# 공백 부분을 없애려면 strip을 사용한다
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [87]:
first, second, third = pieces
print(first)
first + '::' + second + '::' + third

a


'a::b::guido'

In [88]:
# 더 효과적인 방법
'::'.join(pieces)

'a::b::guido'

In [89]:
# 단어가 포함되어 있는지를 알려준다
'guido' in val

True

In [90]:
# 앞에서부터의 위치를 찾아준다
val.index(',')

1

In [91]:
val.index('b')

2

In [92]:
val.index('guido')

6

In [93]:
val.index('NO')
# 없는 단어는 에러가 난다

ValueError: substring not found

In [94]:
val.find('NO')
# 단어가 없으면 오류가 아니라 '-1' 를 리턴한다

-1

In [95]:
# 발생 횟수를 알려준다
val.count(',')

2

In [96]:
val.replace(',', '::')

'a::b::  guido'

In [97]:
val.replace(',', '')

'ab  guido'

### 실습예제 6
위의 val문자열의 , 개수와 문자열의 길이를 합한 값을 출력하시오.

In [99]:
val

'a,b,  guido'

In [98]:
len(val) + val.count(',')

13

### 실습예제 7
위의 val문자열의 '  '(공백)을 찾고 몇 번째 index에 위치해 있는지 나타내시오.

In [101]:
# 앞에서부터의 위치를 찾아준다
val.index(' ')

4

### 정규식

In [104]:
# 정규표현식, 텍스트에서 문자열을 찾는 도구
# regex
# 패턴 매칭, 치환, 분리 기능 등을 제공한다
# 문자열을 분리하는 예로 하나 이상의 스페이스를 의미하는 '\s+'를 사용한다
import re
text = "foo    bar\t baz  \tqux"
re.split(',', text)

['foo    bar\t baz  \tqux']

In [105]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [106]:
# 정규표현식을 컴파일하고 이 객체를 이용하는 방법도 있다
# 반복적으로 사용될 때 편리하고 속도도 빠르다
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [107]:
text

'foo    bar\t baz  \tqux'

In [51]:
regex.findall(text)

['    ', '\t ', '  \t']

In [109]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}' #해당 부분에 맞게 정규식 변수 pattern을 지정

# re.IGNORECASE는 대소문자 구분을 없애준다.
regex = re.compile(pattern, flags=re.IGNORECASE)

In [110]:
# 문자열에서 이 표현식과 일치하는 모든 부분을 찾는다
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

### 실습예제 8
위의 text에서 이메일이 아닌 단어 단위를 찾아 출력하여라.

In [131]:
pattern2 = '[A-Z]{1,10}'
#[대문자가 포함된 모든 문자열]{1개 이상의 개수 반복}
regex2 = regex = re.compile(pattern2, flags=re.IGNORECASE)
regex2.findall(text)

['Dave',
 'dave',
 'google',
 'com',
 'Steve',
 'steve',
 'gmail',
 'com',
 'Rob',
 'rob',
 'gmail',
 'com',
 'Ryan',
 'ryan',
 'yahoo',
 'com']

In [132]:
# sesrch는 만족하는 첫번째 항목만 찾아준다
m = regex.search(text)
print(m)

<_sre.SRE_Match object; span=(0, 4), match='Dave'>


In [133]:
text[m.start():m.end()] #첫번째 항목의 시작지점부터 끝까지를 보여준다

'Dave'

In [134]:
#** match **
#주어진 문자열의 첫글자부터 일치하는 패턴 문자열을 반환 (결과값은 1개)
print(regex.match(text))

<_sre.SRE_Match object; span=(0, 4), match='Dave'>


In [135]:
# 해당하는 패턴을 주어진 문자열로 치환한다
print(regex.sub('REDACTED', text))

REDACTED REDACTED@REDACTED.REDACTED
REDACTED REDACTED@REDACTED.REDACTED
REDACTED REDACTED@REDACTED.REDACTED
REDACTED REDACTED@REDACTED.REDACTED



### 실습예제 9
예제 8에서 찿은 단어들을 'python'으로 치환하여라.

In [136]:
print(regex2.sub('python', text))

python python@python.python
python python@python.python
python python@python.python
python python@python.python



In [41]:
# 패턴을 나누려면, 나눌 각 패턴을 ( )로 묶는다
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

NameError: name 're' is not defined

In [156]:
m = regex.match('wesm@bright.net')
print(m)
m.groups() #m을 정규식 패턴에 맞게 나누어 그룹화하는 함수

<_sre.SRE_Match object; span=(0, 15), match='wesm@bright.net'>


('wesm', 'bright', 'net')

In [157]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [158]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text)) #sub함수를 이용해 정규식의 각 부분에 제목(subject)을 명명할 수 있다.

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



In [159]:
# 매치 그룹에 이름을 줄 수 있다
regex = re.compile(r"""
    (?P<username>[A-Z0-9._%+-]+)
    @
    (?P<domain>[A-Z0-9.-]+)
    \.
    (?P<suffix>[A-Z]{2,4})""", flags=re.IGNORECASE|re.VERBOSE)

In [160]:
m = regex.match('wesm@bright.net')
m.groupdict() #사전형태로 그룹화한다.

{'domain': 'bright', 'suffix': 'net', 'username': 'wesm'}

### pandas의 벡터화된 문자열 함수

In [161]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = Series(data)

In [162]:
data.isnull() #값이 비어있는지 확인하는 isnull()함수

Dave     False
Rob      False
Steve    False
Wes       True
dtype: bool

### 실습예제10
data의 isnull()함수를 호출한 값에 True가 없도록 하는 data2를 만들고 다시 isnull()함수를 호출하여라.

In [38]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': 'Wes@naver.com'}
data = Series(data)
data.isnull()

Dave     False
Rob      False
Steve    False
Wes      False
dtype: bool

In [39]:
data.str.contains('gmail') #문자열에 'gmail'을 포함하고있는지 판별한다.

Dave     False
Rob       True
Steve     True
Wes      False
dtype: bool

In [40]:
pattern

NameError: name 'pattern' is not defined

In [166]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes        [(Wes, naver, com)]
dtype: object

In [170]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Rob      True
Steve    True
Wes      True
dtype: bool

In [171]:
type(matches)

pandas.core.series.Series

In [168]:
matches.str.get(1) #get(index)로 해당 위치의 내용을 불러옴

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [153]:
matches.str[0] #str[index]로도 내용을 불러올 수 있음

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [72]:
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object

## Example: 미국 농무부 음식 데이터

In [42]:
import json
db = json.load(open('data/foods-2011-10-03.json'))
len(db)

FileNotFoundError: [Errno 2] No such file or directory: 'data/foods-2011-10-03.json'

In [43]:
# 각 항목에는 각 음식에 대한 정보를 담는다
db[0].keys()

NameError: name 'db' is not defined

In [75]:
db[0]['nutrients'][0]

{'description': 'Protein',
 'group': 'Composition',
 'units': 'g',
 'value': 25.18}

### 실습예제11
위의 첫 번째 항목의 영양소를 표시한 것 처럼, 식품의 양(portions)을 출력하여라.

In [76]:
nutrients = DataFrame(db[0]['nutrients'])
nutrients[:7]

,description,group,units,value
0,Protein,Composition,g,25.18
1,Total lipid (fat),Composition,g,29.20
2,"Carbohydrate, by difference",Composition,g,3.06
3,Ash,Other,g,3.28
4,Energy,Energy,kcal,376.00
5,Water,Composition,g,39.28
6,Energy,Energy,kJ,1573.00


In [77]:
info_keys = ['description', 'group', 'id', 'manufacturer']
info = DataFrame(db, columns=info_keys)

In [78]:
info[:5]

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,


In [79]:
info

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,
5,"Cheese, mozzarella, part skim milk, low moisture",Dairy and Egg Products,1029,
6,"Cheese, romano",Dairy and Egg Products,1038,
7,"Cheese, roquefort",Dairy and Egg Products,1039,
8,"Cheese spread, pasteurized process, american, ...",Dairy and Egg Products,1048,
9,"Cream, fluid, half and half",Dairy and Egg Products,1049,


In [80]:
# 음식 그룹의 분포를 찾는다
pd.value_counts(info.group)[:10]

Vegetables and Vegetable Products    812
Beef Products                        618
Baked Products                       496
Breakfast Cereals                    403
Fast Foods                           365
Legumes and Legume Products          365
Lamb, Veal, and Game Products        345
Sweets                               341
Fruits and Fruit Juices              328
Pork Products                        328
Name: group, dtype: int64

### 실습예제 12
영양소 자료구조의 그룹의 분포 출력하여라.

In [81]:
''' 
영양소 정보를 분석
'''
nutrients = []

for rec in db: #data cleaning을 위해 영양소 리스트의 id를 같은 음식별로 묶는 함수 rec
    fnuts = DataFrame(rec['nutrients'])
    fnuts['id'] = rec['id']
    nutrients.append(fnuts)

nutrients = pd.concat(nutrients, ignore_index=True) #concat함수를 이용해 기존의 nutrients에 id column을 새로 생성하여 붙임.

In [82]:
nutrients

,description,group,units,value,id
0,Protein,Composition,g,25.180,1008
1,Total lipid (fat),Composition,g,29.200,1008
2,"Carbohydrate, by difference",Composition,g,3.060,1008
3,Ash,Other,g,3.280,1008
4,Energy,Energy,kcal,376.000,1008
5,Water,Composition,g,39.280,1008
6,Energy,Energy,kJ,1573.000,1008
7,"Fiber, total dietary",Composition,g,0.000,1008
8,"Calcium, Ca",Elements,mg,673.000,1008
9,"Iron, Fe",Elements,mg,0.640,1008


In [83]:
nutrients.duplicated().sum() #중복된 값 더함

14179

In [84]:
nutrients = nutrients.drop_duplicates() #중복된 값을 제거

In [85]:
col_mapping = {'description' : 'food',
               'group'       : 'fgroup'}
info = info.rename(columns=col_mapping, copy=False) #col_mapping에서 새로 정한 이름으로 info를 rename
info

,food,fgroup,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,
5,"Cheese, mozzarella, part skim milk, low moisture",Dairy and Egg Products,1029,
6,"Cheese, romano",Dairy and Egg Products,1038,
7,"Cheese, roquefort",Dairy and Egg Products,1039,
8,"Cheese spread, pasteurized process, american, ...",Dairy and Egg Products,1048,
9,"Cream, fluid, half and half",Dairy and Egg Products,1049,


### 실습예제13
위의 col_mapping의 'id'컬럼을 'food_id'로 변경하여라.

In [86]:
col_mapping = {'description' : 'nutrient',
               'group' : 'nutgroup'}
nutrients = nutrients.rename(columns=col_mapping, copy=False)
nutrients

,nutrient,nutgroup,units,value,id
0,Protein,Composition,g,25.180,1008
1,Total lipid (fat),Composition,g,29.200,1008
2,"Carbohydrate, by difference",Composition,g,3.060,1008
3,Ash,Other,g,3.280,1008
4,Energy,Energy,kcal,376.000,1008
5,Water,Composition,g,39.280,1008
6,Energy,Energy,kJ,1573.000,1008
7,"Fiber, total dietary",Composition,g,0.000,1008
8,"Calcium, Ca",Elements,mg,673.000,1008
9,"Iron, Fe",Elements,mg,0.640,1008


In [87]:
ndata = pd.merge(nutrients, info, on='id', how='outer') #id를 키로 해서 외부조인

In [88]:
ndata

,nutrient,nutgroup,units,value,id,food,fgroup,manufacturer
0,Protein,Composition,g,25.180,1008,"Cheese, caraway",Dairy and Egg Products,
1,Total lipid (fat),Composition,g,29.200,1008,"Cheese, caraway",Dairy and Egg Products,
2,"Carbohydrate, by difference",Composition,g,3.060,1008,"Cheese, caraway",Dairy and Egg Products,
3,Ash,Other,g,3.280,1008,"Cheese, caraway",Dairy and Egg Products,
4,Energy,Energy,kcal,376.000,1008,"Cheese, caraway",Dairy and Egg Products,
5,Water,Composition,g,39.280,1008,"Cheese, caraway",Dairy and Egg Products,
6,Energy,Energy,kJ,1573.000,1008,"Cheese, caraway",Dairy and Egg Products,
7,"Fiber, total dietary",Composition,g,0.000,1008,"Cheese, caraway",Dairy and Egg Products,
8,"Calcium, Ca",Elements,mg,673.000,1008,"Cheese, caraway",Dairy and Egg Products,
9,"Iron, Fe",Elements,mg,0.640,1008,"Cheese, caraway",Dairy and Egg Products,


### 실습예제 14
nutrients와 info를 innerjoin한 ndata2를 생성하여라.

In [89]:
ndata.ix[30000] #x번째 행을 나타내는 ix[x]

nutrient                                       Glycine
nutgroup                                   Amino Acids
units                                                g
value                                             0.04
id                                                6158
food            Soup, tomato bisque, canned, condensed
fgroup                      Soups, Sauces, and Gravies
manufacturer                                          
Name: 30000, dtype: object

In [90]:
by_nutrient = ndata.groupby(['nutgroup', 'nutrient']) #groupby()함수를 사용하여 nutgroup에 따른 nutrient로 group화한다.

get_maximum = lambda x: x.xs(x.value.idxmax()) #람다함수는 함수를 지정(직접 만드는 방식), 
                                               #이 함수는 value가 가장 높은 수의 인덱스를 찾아줌 -> idxmax()함수
get_minimum = lambda x: x.xs(x.value.idxmin()) # -> 반대로 가장 낮은 수의 인덱스를 찾아줌 -> idxmin() 함수

max_foods = by_nutrient.apply(get_maximum)[['value', 'food']] #apply (lambda함수) 문법을 이용해 만든 람다 함수를 실행

max_foods.food = max_foods.food.str[:50]

In [91]:
max_foods.ix['Amino Acids']['food']

nutrient
Alanine                           Gelatins, dry powder, unsweetened
Arginine                               Seeds, sesame flour, low-fat
Aspartic acid                                   Soy protein isolate
Cystine                Seeds, cottonseed flour, low fat (glandless)
Glutamic acid                                   Soy protein isolate
Glycine                           Gelatins, dry powder, unsweetened
Histidine                Whale, beluga, meat, dried (Alaska Native)
Hydroxyproline    KENTUCKY FRIED CHICKEN, Fried Chicken, ORIGINA...
Isoleucine        Soy protein isolate, PROTEIN TECHNOLOGIES INTE...
Leucine           Soy protein isolate, PROTEIN TECHNOLOGIES INTE...
Lysine            Seal, bearded (Oogruk), meat, dried (Alaska Na...
Methionine                    Fish, cod, Atlantic, dried and salted
Phenylalanine     Soy protein isolate, PROTEIN TECHNOLOGIES INTE...
Proline                           Gelatins, dry powder, unsweetened
Serine            Soy protein isolate, 